### Summary
Record mean 1 row of the csv file. 

This notebook converts the records into alarms. 

In [1]:
import pandas as pd
import gc # for forcing garbage collection
import time
from alarms import  convertRecordsToAlarms
start = time.time()

In [2]:
input_fname = "formatted-pre-1-months3.csv" 
output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
path = "./data/new/"
output_fname

'formatted-pre-2-months3.csv'

In [3]:
df = pd.read_csv(path + input_fname, low_memory=False, parse_dates=["EventTime"])

# only for testing
# # # df = df.loc[df['SourceName'].isin(["48TIC2040"])]
# # # df.to_csv(path+"test.csv",index = False)
# # # df = pd.read_csv(path + "test.csv", low_memory=False, parse_dates=["EventTime"])

df.info()
# df["EventTime"] = df["EventTime"].apply(lambda d: parse(d))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4899248 entries, 0 to 4899247
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   EventTime    datetime64[ns]
 3   Message      object        
 4   Quality      int64         
 5   Condition    object        
 6   Mask         int64         
 7   NewState     int64         
 8   Status       int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 336.4+ MB


In [4]:
df.head()

,MachineName,SourceName,EventTime,Message,Quality,Condition,Mask,NewState,Status
0,PHD47B,47PDI1747,2019-04-01 01:40:53,47PDI1747 E136 FARK BASINC ACK,0,IOP,194,7,0
1,PHD47B,47FLI-042,2019-04-01 01:40:09,47FLI-042 C-118 TEPE ACK,0,IOP,194,7,0
2,PHD47B,47FLI-042,2019-04-01 01:40:09,47FLI-042 C-118 TEPE ACK,0,CNF,194,7,0
3,PHD47B,47PIC1505,2019-04-01 01:40:09,47PIC1505 NAF.SPLT.TEPE ACK,192,LL,194,7,0
4,PHD47B,47PI123ABC,2019-04-01 01:10:55,47PI123ABC 47F-1 FUEL GAS PRES ACK,192,HHH,194,7,0


In [5]:
alarms = []
print("Total Number of Sources :  ", len(df["SourceName"].unique()))
print("Staring Processing records of each source : ", end= " ")
index = 0 # only for printing 
for sname in df["SourceName"].unique():
    records =  df.loc[df['SourceName'].isin([sname])]
    records = records.to_dict(orient="records")
#     convertRecordsToAlarms()
    temp_alarms = convertRecordsToAlarms(records)
    print("({},{},{})".format(index,sname, len(temp_alarms)), end=" ")
    alarms += temp_alarms # concat list
    index += 1
    
#     if index == 2: # just for debugging
#         break
print("\n\n Done: All records are converted to to alarms.")

Total Number of Sources :   974
Staring Processing records of each source :  (0,47PDI1747,4646) (1,47FLI-042,452) (2,47PIC1505,591) (3,47PI123ABC,9) (4,48LI013,182) (5,47FIC034,13206) (6,47PIC2033A,607) (7,47XL015-ANN,459) (8,47LI003,942) (9,47TXI1594,76) (10,48TIC034,667) (11,47TI013,6) (12,47TI1506,167) (13,47XL016-ANN,452) (14,47FLI-037,258) (15,47PI1701,22) (16,48XL006-ANN,82) (17,48XL002-ANN,1443) (18,47FIC016,35) (19,47PI1710,106) (20,19FI108A,67) (21,47PDI003,509) (22,47TI1512A,320) (23,47PDI1749,328) (24,48LI015,354) (25,47TI931A,735429) (26,48FIX2040,80191) (27,48TIC2027,80190) (28,48AAHH004-ANN,3632) (29,48AI004,3618) (30,47FI1568,4405) (31,47TI1713,15805) (32,47_AMPSB_PMCC1L,21) (33,47TI1723,425) (34,47PIC2038,45) (35,47TI260,8) (36,47FIC1515,112) (37,47TI1765,19) (38,47TI1727,31) (39,47PDI1746,7411) (40,47FIC1522A,412) (41,47_AMPSA_PMCC1L,79) (42,48FALL2003-ANN,329) (43,47FIC021,21) (44,19LAH101-ANN,29) (45,47LIC1509,383) (46,48XL012-ANN,77) (47,47_AMPSC_PMCC1L,60) (48,47TI

In [6]:
df2 = pd.DataFrame(alarms)
df2.to_csv(path + output_fname, index = False)
print("Processing Time = {} minutes".format((time.time()-start)/60.0))
df2

Processing Time = 2.7075292150179546 minutes


,MachineName,SourceName,Message,Quality,Condition,Mask,NewState,Status,StartTime,EndTime,EndMessage
0,PHD47B,47PDI1747,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP,0,IOP,249,3,0,2019-04-01 00:04:52,2019-04-01 00:04:55,47PDI1747 E136 FARK BASINC PV = 0.53 Kg IOP Re...
1,PHD47B,47PDI1747,47PDI1747 E136 FARK BASINC PV = 0.53 Kg IOP,0,IOP,201,3,0,2019-04-01 00:04:58,2019-04-01 00:19:49,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP Re...
2,PHD47B,47PDI1747,47PDI1747 E136 FARK BASINC ACK,0,IOP,194,7,0,2019-04-01 00:13:29,2019-04-01 00:19:57,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP Re...
3,PHD47B,47PDI1747,47PDI1747 E136 FARK BASINC PV = 0.50 Kg IOP,0,IOP,249,3,0,2019-04-01 00:19:53,2019-04-01 00:22:30,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP Re...
4,PHD47B,47PDI1747,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP,0,IOP,201,3,0,2019-04-01 00:20:19,2019-04-01 00:23:52,47PDI1747 E136 FARK BASINC PV = 0.52 Kg IOP Re...
...,...,...,...,...,...,...,...,...,...,...,...
2427028,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 6.2 C VEL+,192,VEL+,193,3,1,2019-06-27 18:38:34,2019-06-27 18:38:37,48TIC2040 DESUPERHEATER PV = 7.2 C VEL+ Recover
2427029,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 63.1 C VEL+,192,VEL+,193,3,1,2019-06-27 18:38:59,2019-06-27 18:39:01,48TIC2040 DESUPERHEATER PV = 119.6 C VEL+ Recover
2427030,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 119.6 C IOP,0,IOP,201,3,1,2019-06-27 18:39:02,2019-06-27 18:39:42,48TIC2040 DESUPERHEATER PV = 3.0 C IOP Recover
2427031,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 4.3 C VEL-,192,VEL-,193,3,1,2019-06-27 18:39:53,2019-06-27 18:39:55,48TIC2040 DESUPERHEATER PV = 6.1 C VEL- Recover
